In [10]:
#Imports
import pandas as pd
import numpy as nd
import os

In [11]:
#Input columns to use from csv sheets
MASTER_COLS_TO_USE = ['azSeries','azVmName','vCPU','vMem','vSAPS','Enabled']
INPUT_COLS_TO_USE = ['vmID','vCPU','vMem','DimDataInstance']

#Weight in error function for each of the mapping parameters
wvCPU = 1
wSAPS = 0
wMem = 1

#MasterData, Input file path
Separator = ";"
MASTER_CSV_PATH = os.path.join('data','ALJ-vmMasterData.csv')
DATA_CSV_PATH = os.path.join('data', 'ALJ-Optimize.csv')

#Path to output file
OUT_CSV_PATH = os.path.join('data', 'ALJ-mapDataOut.csv')

#read sap masterdata
sapData = pd.read_csv(MASTER_CSV_PATH, usecols = MASTER_COLS_TO_USE, sep = Separator)
#Only take the enabled vm's into account
sapData = sapData[sapData['Enabled']]
print(sapData)

   azSeries  azVmName  vCPU  vMem   vSAPS  Enabled
0      Esv3     E2sv3     2    16    2178     True
1      Esv3     E4sv3     4    32    4355     True
2      Esv3  E8-4s_v3     4    64    4378     True
3      Esv3    E8s_v3     8    64    8710     True
4      Esv3   E16s_v3    16   128   17420     True
5      Esv3   E20s_v3    20   160   21775     True
6      Esv3   E32s_v3    32   256   34840     True
7      Esv3   E38s_v3    48   384   52512     True
8      Esv3  E64is_v3    64   432   70050     True
9      Esv3   E64s_v3    64   432   70050     True
10     Dsv3    D2s_v3     2     8    2178     True
11     Dsv3    D4s_v3     4    16    4355     True
12     Dsv3    D8s_v3     8    32    8710     True
13     Dsv3   D16s_v3    16    64   17420     True
14     Dsv3   D32s_v3    32   128   34840     True
15     Dsv3   D48s_v3    48   192   52224     True
16     Dsv3   D64s_v3    64   256   69680     True
32        M      M8ms     8   219    8616     True
33        M     M16ms    16   4

In [16]:
#read input data
mapData = pd.read_csv(DATA_CSV_PATH, usecols = INPUT_COLS_TO_USE, sep = Separator)
INPUT_COLS_TO_USE = ['vmID','vCPU','vMem','DimDataInstance']

mapData['vSAPS'] = 0
mapData['targetVM'] = 'tbd'

In [17]:
print(mapData)

              vmID  vCPU  vMem DimDataInstance  vSAPS targetVM
0       aljerpdev      8  49.4         D8s_v3       0      tbd
1      aljerpprd1      8  65.9         D8s_v3       0      tbd
2      aljerpprd2      4  65.9         D4s_v3       0      tbd
3     aljerpprda1     16  65.9        D16s_v3       0      tbd
4    aljerpprda10     16  65.9        D16s_v3       0      tbd
..             ...   ...   ...             ...    ...      ...
157     aljnw9ci2      4  16.4         D4s_v3       0      tbd
158     aljnw9db1      8  32.9         D8s_v3       0      tbd
159     aljnw9db2      8  32.9         D8s_v3       0      tbd
160   aljnw9pasfm      4   8.1         D4s_v3       0      tbd
161     ALJSAPGUI      8   8.6         D8s_v3       0      tbd

[162 rows x 6 columns]


In [19]:
 #Determine min / max values from masterData for normalization
minvCPU  = sapData['vCPU'].min()
minvMem  = sapData['vMem'].min()
minvSAPS = sapData['vSAPS'].min()
maxvCPU  = sapData['vCPU'].max()
maxvMem  = sapData['vMem'].max()
maxvSAPS = sapData['vSAPS'].max()

#Normalization (value - min ) / (max - min)
#Normalization MasterData
sapData['nvCPU']  = (sapData['vCPU']  - minvCPU)  / (maxvCPU - minvCPU)
sapData['nvMem']  = (sapData['vMem']  - minvMem)  / (maxvMem - minvMem)
sapData['nvSAPS'] = (sapData['vSAPS'] - minvSAPS) / (maxvSAPS - minvSAPS)

#Iterate
for index, row in mapData.iterrows():
    sapDataTemp = sapData
    #print(f"vmName : {row['vmID']}", f"vCPU : {row['vCPU']}", f"vMem : {row['vMem']}", f"vSAPS : {row['vSAPS']}", sep = " / ")
          
    #Add Target colums
    sapDataTemp['xCPU']  = row['vCPU']
    sapDataTemp['xMem']  = row['vMem']
    sapDataTemp['xSAPS'] = row['vSAPS']
    
    #Normalization Target Data
    sapDataTemp['nxCPU']  = (sapDataTemp['xCPU']  - minvCPU)  / (maxvCPU - minvCPU)
    sapDataTemp['nxMem']  = (sapDataTemp['xMem']  - minvMem)  / (maxvMem - minvMem)
    sapDataTemp['nxSAPS'] = (sapDataTemp['xSAPS'] - minvSAPS) / (maxvSAPS - minvSAPS)

    #Error distance calculation
    sapDataTemp['CPUError']  = (sapDataTemp['nvCPU'] - sapDataTemp['nxCPU'])**2
    sapDataTemp['MemError']  = (sapDataTemp['nvMem'] - sapDataTemp['nxMem'])**2
    sapDataTemp['SAPSError'] = (sapDataTemp['nvSAPS'] - sapDataTemp['nxSAPS'])**2
    sapDataTemp['tError']    = (wvCPU * sapDataTemp['CPUError'] + wMem * sapDataTemp['MemError'] + wSAPS * sapDataTemp['SAPSError'])/(wvCPU+wMem+wSAPS)

    #Select lowest error distance
    result = sapDataTemp.loc[sapDataTemp['tError'].idxmin()]
    
    #Update mapping column
    mapData.at[index, 'targetVM'] = result['azVmName']
    mapData.at[index, 'Error']    = result['tError']
    mapData.at[index, 'tCPU']     = result['vCPU']
    mapData.at[index, 'tMem']     = result['vMem']
    mapData.at[index, 'tSAPS']    = result['vSAPS']
   


In [20]:
print(mapData)

#Write to csv file
mapData.to_csv(OUT_CSV_PATH, header=True)

              vmID  vCPU  vMem DimDataInstance  vSAPS  targetVM         Error  \
0       aljerpdev      8  49.4         D8s_v3       0      DS13  1.443775e-06   
1      aljerpprd1      8  65.9         D8s_v3       0    E8s_v3  1.196517e-07   
2      aljerpprd2      4  65.9         D4s_v3       0  E8-4s_v3  1.196517e-07   
3     aljerpprda1     16  65.9        D16s_v3       0   D16s_v3  1.196517e-07   
4    aljerpprda10     16  65.9        D16s_v3       0   D16s_v3  1.196517e-07   
..             ...   ...   ...             ...    ...       ...           ...   
157     aljnw9ci2      4  16.4         D4s_v3       0    D4s_v3  5.303121e-09   
158     aljnw9db1      8  32.9         D8s_v3       0    D8s_v3  2.684705e-08   
159     aljnw9db2      8  32.9         D8s_v3       0    D8s_v3  2.684705e-08   
160   aljnw9pasfm      4   8.1         D4s_v3       0    D4s_v3  2.068549e-06   
161     ALJSAPGUI      8   8.6         D8s_v3       0    D8s_v3  1.814861e-05   

     tCPU  tMem    tSAPS  
